Initialization

In [ ]:
from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp,Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.quantum_info import Statevector,random_statevector
import math
import json
import os

%run config.py

Load the directories and identify the project inside them.

In [ ]:
def rotfitness(parameters, *args):
	qbits=int(len(parameters)/3)

	qc = QuantumCircuit(qbits)
	for i in range(qbits):
		qc.rx(float(parameters[3*i]), i)
		qc.ry(float(parameters[3*i+1]), i)
		qc.rz(float(parameters[3*i+2]), i)
	
	# print (qc.draw())
	qc=qc.reverse_bits()

	complexArray = []
	complexArray.append(complex(1,0))
	for i in range(2**qbits-1):
		complexArray.append(complex(0,0))

	sv = Statevector(complexArray)
	sv = sv.evolve(qc)

	outlistcomplex = sv.data.tolist()
	# print(outlistcomplex)
	# print (args)

	sigma = 0.0
	for i in range(len(outlistcomplex)):
		sigma  += abs( args[2*i] - outlistcomplex[i].real)
		sigma  += abs( args[2*i+1] - outlistcomplex[i].imag)

	sigma = sigma/len(outlistcomplex)
	# print (sigma)
	return sigma

In [ ]:
def guessrotations(target):
	# target is a statevector
	length = len(target)
	qbits = int(math.log2(length)/2)
	# print (qbits)
	initial_guess = [1.0,] * 3*qbits
	
	result = minimize(rotfitness, initial_guess, args=tuple(target))
	return result.x

In [ ]:
# List all the directories, only the directories
dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
activeproj = {}
base = ""
for d in dirs:
	if os.path.exists(d+"/active"):
		with open(d+"/active","r") as f:
			ptype = f.read().strip()
			activeproj[d] = ptype
		if os.path.exists(d+"/base"):
			base = d
			if ptype != "qiskit":
				print("Warning: base project is not of type qiskit")
		
# print("Active projects: ",activeproj)
# print("Static data: ",staticdata)
# print("Base project: ",base)

Create a set of inputs for the test cases. They should be normalized values, let's use the qiskit random_statevector function to generate them.

In [ ]:
if fundamental:
	inputs = []
	outlist=[[1.0,0.0]]
	for i in range(2**qbits-1):
		outlist.append([0.0,0.0])
	inputs.append(outlist)
else:
	if staticdata:
		with open('inputs.json', 'r') as f:
			inputs = json.load(f)
	else:
		inputs = []
		for i in range(numiter):
			sv=random_statevector(2**qbits)
			outlist=[]
			for i in range(len(sv)):
				outlist.append([sv[i].real,sv[i].imag])
			inputs.append(outlist)


		with open('inputs.json', 'w') as f:
			json.dump(inputs, f)

Convert the inputs to the format expected by the single project.

In [ ]:
for p in activeproj:
	if activeproj[p] == "qiskit":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= [j[0], j[1]]
				inputJson.append(val)
			inputsJson.append( inputJson )

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
	if activeproj[p] == "bmqsim":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= {"Real": j[0], "Imag": j[1]}
				inputJson.append(val)
			inputsJson.append({ "Vector": inputJson })

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		inputValues=[]
		for i in inputs:
			for j in i:
				inputValues.append(j[0])
				inputValues.append(j[1])

		with open(p+'/inputs.txt', 'w') as f:
			for item in inputValues:
				f.write("%s\n" % item)

Run all the projects

In [ ]:
for p in activeproj:	
	cwd = os.getcwd()
	os.chdir(p)
	if os.path.exists("circuit.py"):
		%run circuit.py
	elif os.path.exists("circuit.ipynb"):
		%run circuit.ipynb
	os.chdir(cwd)

OUTPUT checks the output of the projects and compares them with the pivot project.


In [ ]:
passed = True
for p in activeproj:
	if activeproj[p] == "qiskit" and p!=base:
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			qiskitoutputs2 = json.load(f)

		if len(qiskitoutputs) != len(qiskitoutputs2):
			print("Warning: Different number of outputs")

		qvars = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			qiskitoutput2 = qiskitoutputs2[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				qreal2 = float(qiskitoutput2[j][0])
				qimag2 = float(qiskitoutput2[j][1])
				qvars[j][0] += pow(qreal - qreal2, 2)
				qvars[j][1] += pow(qimag - qimag2, 2)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False	

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))
			
	if activeproj[p] == "bmqsim":
		
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			bmqsimoutputs = json.load(f)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]["Vector"]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j]["Real"])
				bimag = float(bmqsimoutput[j]["Imag"])
				bmvars[j][0] += pow(qreal - breal, 2)
				bmvars[j][1] += pow(qimag - bimag, 2)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False	

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.txt', 'r') as f:
			bmqsimoutputs = []
			for k in range(len(qiskitoutputs)):
				bmqsimoutput=[]
				for i in range(2 ** qbits):
					line = f.readline()
					vreal=float(line)
					line = f.readline()
					vimm=float(line)
					bmqsimoutput.append([vreal,vimm])
				bmqsimoutputs.append(bmqsimoutput)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j][0])
				bimag = float(bmqsimoutput[j][1])
				bmvars[j][0] += pow(qreal - breal, 2)
				bmvars[j][1] += pow(qimag - bimag, 2)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False	

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

In [ ]:
if not quiet:
	if passed:
		print("Passed")
	else:
		print("Failed")